# 고금계 과제 1

20249433 최재필

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path

In [6]:
CWD = Path('.').resolve()
DATA_DIR = CWD / 'data'

In [7]:
from fndata import FnStockData

In [14]:
from fndata import FnMarketData

## Load data

In [8]:
fndata_path = DATA_DIR / '고금계과제1_v3.3_201301-202408.csv'

In [15]:
fnmkt_path = DATA_DIR / '고금계과제_시장수익률_201301-202408.csv'

In [20]:
rf_path = DATA_DIR / '통안채1년물_월평균_201301-202408.csv'

In [9]:
# 데이터 모듈을 생성하며 기본 전처리들을 수행합니다.
fn = FnStockData(fndata_path)

In [16]:
# 데이터 모듈을 생성하며 기본 전처리들을 수행합니다.
fnmkt = FnMarketData(fnmkt_path)

In [10]:
# 사용 가능한 데이터를 확인합니다.
fn_items = fn.get_items()
fn_items

array(['종가(원)', '수정계수', '수정주가(원)', '수익률 (1개월)(%)', 'FnGuide Sector',
       '거래정지여부', '관리종목여부', '보통주자본금(천원)', '자본잉여금(천원)', '이익잉여금(천원)',
       '자기주식(천원)', '이연법인세부채(천원)', '매출액(천원)', '매출원가(천원)', '이자비용(천원)',
       '영업이익(천원)', '총자산(천원)', '기말발행주식수 (보통)(주)'], dtype=object)

In [11]:
# 분석 기간의 데이터 유니버스를 확인합니다. (금융업종, 거래정지, 관리종목 제외)
univ_list = fn.get_universe()
univ_list

array(['A000020', 'A000040', 'A000050', ..., 'A001260', 'A019660',
       'A023460'], dtype=object)

In [13]:
# 모든 데이터를 불러옵니다.

df = fn.get_data()

In [19]:
mkt = fnmkt.get_data(format='wide')

mkf2000 = mkt['MKF2000']
krx300 = mkt['KRX 300']

In [40]:
rf = pd.read_csv(rf_path)

In [41]:
rf.columns = ['date', 'rf']

In [42]:
rf.set_index('date', inplace=True)

In [43]:
rf.index = pd.to_datetime(rf.index, format='%Y/%m') + pd.offsets.MonthEnd(0)

In [45]:
rf_m = (1 + rf / 100) ** (1 / 12) - 1
rf_m

,rf
date,
2013-01-31,0.002239
2013-02-28,0.002214
2013-03-31,0.002149
2013-04-30,0.002125
2013-05-31,0.002125
...,...
2024-04-30,0.002781
2024-05-31,0.002780
2024-06-30,0.002751


## 1. Mkt Cap

= 기업규모 = 시장가치 = 시가총액 = 보통주 주가 * 발행주식수

## 2. B/M

자기자본 장부가치 / 시장가치

### 2.1 Book 계산

= 자기자본 장부가치 = t-1년 12월말 보통주 자본금 + 자본잉여금 + 이익잉여금 + 자기주식 + 이연법인세 부채

** 장부가치 음수인 자본잠식 상태의 기업들 분석에서 제외

### 2.2 B/M 계산

## 3. 수익성

= ROE = Net Income / Equity 

### 3.1 Net Income

= t-1년 12월 말의 매출액 - 매출원가 - 이자비용 - 판매및관리비 차감한 영업이익 

### 3.2 ROE 계산

NI, Equity 둘 다 t-1년 12월 말 기준

## 4. 자본투자

= (t-1년 12월 총자산 - t-2년 12월 총자산) / t-2년 12월 총자산 = 즉, 총자산의 증가율

## 5. UMD

= Up Minus Down = (전월말 주가 - 1년전 월말 주가 ) / 1년전 월말 주가

** 모멘텀의 경우 1개월마다 리밸런싱, 상위 30%가 winner / 하위 30%가 loser

## 6. STR

= reversal 

최근 월의 수익률을 기반으로 구축 ? 